# Mapping and contextualizing historical features with loc.gov API

Digital mapping has become an increasingly accessible and valuable complement to traditional interpretive narratives. Working with spatially-referenced data offers exciting possibilities for placed-based scholarship, outreach, and teaching. It’s also a perfect avenue for interdisciplinary collaboration — between, say, humanities researchers new to GIS and spatial scientists who’ve been using it for decades.

Embedded within digital collections available from the Library of Congress website are geographic data, including the locations of items and their local contexts. We can gather those data programmatically (using Python in this case) and plot them on a map, like so:

!["mapping HAER points"](https://raw.githubusercontent.com/charlie-moffett/data-exploration/master/files/haer_tutorial_screenshot.JPG)

The story of these data would be incomplete, however, without a critical understanding of the history behind their collection and stewardship. In this tutorial, we demonstrate how loc.gov JSON API users can find and store spatial information from Library content with an awareness toward data quality, provenance, and why this broadened scope is important for informing research projects at the Library. 

__Rights and access__

Rights and restrictions, including copyright, affect how you can use images, particularly if you want to publish, display, or otherwise distribute them. You can read more about copyright and other restrictions that apply to the publication/distribution of images from the Prints & Photographs Division (P&P) at this link: https://www.loc.gov/rr/print/195_copr.html

The records in the case study that follows were created for the U.S. Government and are considered to be in the public domain. It is understood that access to this material rests on the condition that should any of it be used in any form or by any means, the author of such material and the Historic American Engineering Record of the Heritage Conservation and Recreation Service at all times be given proper credit.

__Data quality__

Consistency and accuracy of geographic information stored with digital content on the Library of Congress website varies across and within collections. This tutorial was designed as a method of exploring existing spatial references for items. Finding and analyzing meaningful patterns from secondary data typically requires additional data corrections, contexts, and geocoding frameworks to ensure optimal coverage and accuracy. The same applies to spatial data analysis involving digital collections at the Library of Congress; in any case, time spent analyzing and interpeting data is mostly spent cleaning and grappling with the data. Rather than gloss over it, here we elect to revel in that critical stage of research to both demonstrate a particular subset of techniques, and to appreciate its nuances and implications.

__Case Study__

As a graduate student of applied urban science, I was inspired at the outset of my internship with LC Labs to discover content about the built environment across US cities on the Library website. What I found was an expansive dataset of digitized photographs, drawings and reports recognized collectively as the HHH, which includes material from three programs:

- Historic American Buildings Survey (HABS)
- Historic American Engineering Record (HAER)
- Historic American Landscapes Survey (HALS)

More about HHH: https://www.loc.gov/collections/historic-american-buildings-landscapes-and-engineering-records/about-this-collection/

*Browsing the collection:*

![How to search HHH](https://raw.githubusercontent.com/charlie-moffett/data-exploration/master/files/hhh_search_screenshot.gif)


*Example of material found in the collection:* 

View of the uptown platform at 79th Street. Photo by David Sagarin for the Historical American Engineering Record, Library of Congress, Prints and Photographs Division, August 1978.

![Interborough subway, NYC](https://raw.githubusercontent.com/charlie-moffett/data-exploration/master/files/haer_subway_screenshot.jpg)

__HAER__

I decided to dig a bit deeper into the engineering record because it appeared to have the best coverage of the three for spatial references. The Historic American Engineering Record, or HAER, was established in partnership by the National Park Service, the American Society of Civil Engineers and the Library of Congress in 1969. There are more than 10,000 HAER surveys of historic sites and structures related to engineering and industry. The collection is an ongoing effort with established guidelines for documentation -- HAER was created to preserve these structures through rule-based documentation, and those documents have in turn been preserved through time.

Read more about HAER Guidelines here: https://www.nps.gov/hdp/standards/haerguidelines.htm

As happens often with classifications, there are records that could rightly be placed under more than one program. There are many sites documented under the Historic American Building Surveys (HABS) - like bridges, for example - that if recorded today would be assigned under HAER.  Bridges are one key example. Before HAER was established, engineering-related structures, manufacturing and industrial sites, processes, watercraft, bridges, vehicles, etc. were documented under HABS. What I've aimed to develop here, as a result of both my own investigation as well as conversations with Library staff, is a flexible and reproducible approach to looking at the implications for scholarship of changes to preservation over time.  

Geography was actually the motivating factor behind how these materials were originally organized by the NPS and the LC. In the 1930s, HABS surveyors were organized into district offices that documented one or more states. The materials that they created usually (not always, which is a different problem) included statecounty/city.  While the Park Service played a leading role in documenting geographic data, the Library of Congress also shaped the material in the process of archiving the surveys. 

The LC uses two systems to organize HABS/HAER/HALS documentation: The newer system uses the survey number as the call number. The older system assigned each survey a call number based on its location (state/county/city). Ex: [HABS AL-654](http://loc.gov/pictures/item/al0003/) has the call number: `HABS ALA,1-PRAVI.V,1-`  

- 1 = Autauga County. (Each state's counties are assigned numbers in alphabetical order.)
- PRAVI = Prattville.
- .v = in the vicinity of a given city/town.
- 1- = first place in the vicinity of Prattville surveyed.

Places documented in rural and unincorporated areas (and even some in urban areas) have good site maps/UTM/decimal degree data; some don't. When places can't be located, even the vicinity, or in the rare cases when an address is restricted, city centroid points are used. The National Park Service's [Cultural Resources GIS Program](https://www.nps.gov/crgis/) is currently working on a project to create an enterprise dataset that includes all HABS/HAER/HALS surveys, as they've done for the National Register of Historic Places.

The NPS guidelines for surveys didn't initially include spatial data in the way that it exists today. Good site maps are the best data available for surveys from the 1930s. HABS (and HAER) guidelines were later updated to request Universal Transverse Mercator (UTM) coordinates, a global system of grid-based mapping references. All three programs now ask for decimal degree data (in order to comply with the NPS Cultural Resource Spatial Data Transfer Standards), though they still receive data in UTM (and in some cases no geographic reference at all).

Data transmitted to the Library of Congress by contributor Justine Christianson, a HAER Historian with the National Park Service, is particularly rich for our purpose of visualizing the spatial distribution of collection items. Over the past few years, she has reviewed all of the HAER records to index them and assign decimal degree coorinates. Because she has often been involved in finalizing HAER documentation before it goes to the Library she is often listed as contributor in the record metadata. As I've gathered from speaking with Mary McPartland, Collections Manager at NPS, Justine has created spatial data (not to mention other data improvements) for many more HAER surveys than her name is attached to. The subset chosen for this tutorial, then, reflects a certain signature of her involvement in developing standards of documentation, verifying historical reports, and performing scholarly research on material in the collection - nearly 1,500 items with accurate latitude and longitude attributes polished and preserved.

# Tutorial

The following guide will demonstrate how to plot a selection of items from the Historic American Engineering Record (HAER) on a map. With minor changes, the same process of spatial data extraction and visualization could be applied to other digital collections containing explicit geographic information at the Library of Congress.

__Loading in packages__

The recommended convention in Python's own documentation is to import everything at the top, and on separate lines. For this tutorial, we'll be importing three packages into the notebook:

1) To get our data from the digitzed HAER collection, we'll the `requests` Python module to access the loc.gov JSON API.

2) Reading in coordinates means our data needs to be re-organized - a task for the popular analysis package, `pandas`. 

3) Finally, we'll do our visualization with `folium` to plot the locations on an interactive Leaflet map.

Folium is a Python wrapper for a tool called leaflet.js. With minimal instructions, it does a bunch of open-source Javascript work in the background, and the result is a mobile-friendly, interactive 'Leaflet Map' containing the data of interest.

In [2]:
import requests
import pandas as pd
import folium

__Gathering item geography__

Getting up to speed with use of the loc.gov JSON API and Python to access the collection was a breeze, thanks to data exploration resources from former LC Labs resident Laura Wrubel, Software development librarian at GWU.

Grab more tips for loc.gov JSON API calls and URL parameterization from Laura's 'Accessing images for analysis' notebook:
https://github.com/lwrubel/data-exploration/blob/master/Accessing%20imagebs%20for%20analysis.ipynb

In [4]:
# Many of the prints & photographs in HAER are tagged with geographic coordinates ('latlong')
# Using the requests package we imported, we can easily 'get' data for an item as JSON and parse it for our latlong:

get_any_item = requests.get("https://www.loc.gov/item/al0006/?fo=json")
print('latlong: {}'.format(get_any_item.json()['item']['latlong']))

latlong: 32.45977,-86.47767


In [4]:
# To retrieve this sort of data point for a set of search results, we'll first use Laura's get_image_urls function. 
# This will allow us to store the web address for each item in a list, working through the search page by page.

def get_image_urls(url, items=[]):
    '''
    Retrieves the image_ruls for items that have public URLs available. 
    Skips over items that are for the collection as a whole or web pages about the collection.
    Handles pagination. 
    '''
    # request pages of 100 results at a time
    params = {"fo": "json", "c": 100, "at": "results,pagination"}
    call = requests.get(url, params=params)
    data = call.json()
    results = data['results']
    for result in results:
        # don't try to get images from the collection-level result
        if "collection" not in result.get("original_format") and "web page" not in result.get("original_format"):
            # take the last URL listed in the image_url array
            item = result.get("id")
            items.append(item)
    if data["pagination"]["next"] is not None: # make sure we haven't hit the end of the pages
        next_url = data["pagination"]["next"]
        #print("getting next page: {0}".format(next_url))
        get_image_urls(next_url, items)
        
    return items

To demonstrate with our subset of HAER listed under 'Justine Christianson', I'll use a search that targets items from HAER with the her name listed as the contributor.

In [6]:
url = "https://www.loc.gov/search/?fa=contributor:christianson,+justine&fo=json"
# This is the base URL we will use for the API requests we'll be making as we run the function.

Now we can apply Laura's get_image_urls function to our search results URL, formatted in JSON, to get a list of image URLs: 

In [7]:
# retrieve all image URLs from the search results and store in a variable called 'image_urls'
image_urls = get_image_urls(url, items=[])

# how many URLs did we get?
len(image_urls)

1472

In [16]:
# to save on a little time, let's see what the last 100 look like
img100 = image_urls[200:300]

len(img100)

100

In [17]:
# create an empty set to store our latlongs
# storing in a set rather than a list eliminates any potential duplicates
spatial_set = set()

# the parameters we set for our API calls taken the first function
p1 = {"fo" : "json"}

# loop through the item URLs
for img in img100:
    
    # make HTTP request to loc.gov API for each item
    r = requests.get(img, params=p1)
    
    # extract only from items with latlong attribute
    try:
        
        # expose in JSON format
        data = r.json()
        
        # parse for location
        results = data['item']['latlong']
        
        # add it to our running set
        spatial_set.add(results)
        
    # skip anything with missing 'latlong' data
    except:
        
        # on to the next item until we're through
        pass
    
# show us the data!
spatial_set

{'20.027285,-155.821541',
 '20.027762,-155.822901',
 '21.5690150833522,-164.262295188554',
 '30.327681,-81.654645',
 '32.4159199,-107.5766925',
 '34.062341,-116.380791',
 '34.0812281,-82.4837404',
 '35.9312126,-84.39011115',
 '37.002572,-76.311215',
 '37.406428,-122.034588',
 '37.7513338695077,-119.141566599457',
 '37.7620889746898,-119.860731072203',
 '37.807415,-122.420496',
 '37.8905658845769,-119.846371023279',
 '38.077236,-122.097628',
 '38.291201,-76.814178',
 '38.576216,-77.17956176714539',
 '38.577007,-77.17869035906114',
 '38.585656,-77.16905609368388',
 '38.587391,-77.18042778833892',
 '38.590441,-77.1680864793472',
 '38.591214,-77.17302354049494',
 '38.591331,-77.17252192556298',
 '38.594948,-77.17862427942757',
 '38.684596,-77.131794',
 '38.695772,-76.168949',
 '38.712275,77.036903',
 '38.9001118,-77.0163653',
 '38.931502,-74.92552',
 '38.990624,-77.028595',
 '39.284561,-76.596305',
 '39.477233,-77.7407',
 '39.60303,-81.15734',
 '39.648611,-75.955833',
 '39.794447,-75.57244

In [18]:
# how many unique data points were we able to gather?
len(spatial_set)

75

*Pausing for reflection*

So out of the sample of 100 HAER item URLs that we looped through to pull out spatial references, we ended up with a set of 75 latitude and longitude pairs. Not bad! This is certainly not perfect as far as data coverage is concerned, but given what we learned earlier about the lineage of preservation with this collection and dynamics of stewardship, I feel as though we have enough information for a meaningful demonstration and reasonable confidence in the quality of that data to proceed with the dive.

Something to notice however is how these data are currently formatted. Each latitude and longitude pair is glued together as a single string. This isn't how Folium will want to read in coordinates, so as a next step we'll need to rework them a bit before we get to mapping.

__Data manipulation__

We've mined out the locations of a digital subset from the HAER collection. Now we'll restructure it with the popular `pandas` package. 

In [19]:
# convert latlong set to list
latlong_list = list(spatial_set)

# convert list to pandas dataframe
df = pd.DataFrame(latlong_list)

# split coordinates into two columns
df = df[0].str.split(',', expand=True)

# rename columns with latitude and longitude
df = df.rename(columns={0:'latitude', 1:'longitude'})

# what's the dataframe look like?
df

,latitude,longitude
0,44.811296,-86.056505
1,38.585656,-77.16905609368388
2,38.695772,-76.168949
3,39.648611,-75.955833
4,43.437466,-85.665291
5,37.7513338695077,-119.141566599457
6,38.712275,77.036903
7,39.284561,-76.596305
8,43.01620369,-74.917545182
9,38.291201,-76.814178


*Pausing for reflection*

An interesting thing to note from the dataframe is the number of decimal places across our data points. Some of the coordinates are way more precise than others! This provides us with another glimpse of how changes in technology and methodology over the years can leave their trace in the digital footprint of a collection. I don't suspect that this discrepancy will impact our ability to check out the spatial distribution of the material however.

At this stage you could export your tables of coordinates to combine with existing projects, visualize with other software, etc. I've "commented out" the following line since I already have a copy of "haer_sample.csv" on my machine, but if you've downloaded this notebook and are following along cell by cell, just remove the pound sign and run it to export the file for yourself.

In [20]:
# df.to_csv('haer_sample.csv')

Since we're working in a Jupyter notebook, we can just read back in the .CSV file and make a map right here.

In [21]:
# convert spreadsheet to pandas dataframe
latlong_df = pd.read_csv('files/haer_sample.csv', usecols=[1,2])

# Geovisualization
As we called out at the start of the tutorial, the open-source tool `folium` builds on our earlier data wrangling with `pandas` and the mapping strengths of the Leaflet.js library to create an interactive experience.

In [32]:
# convert pandas dataframe back to a list for folium
latlong_list = latlong_df.values.tolist()

# picking a spot in the midwest to center our map around
COORD = [35.481918, -97.508469]

# uses lat then lon - the bigger the zoom number, the closer in you get
map_haer = folium.Map(location=COORD, zoom_start=3, 
tiles='cartodbpositron', width=640, height=480)

# add a marker to the base leaflet map for every latlong pair in our list
for i in range(len(latlong_list)): 
    folium.CircleMarker(latlong_list[i], radius=1, color='#0080bb', fill_color='#0080bb').add_to(map_haer) 

In [35]:
# calls the map to display
inline_map()

In [36]:
def embed(fmaps, width='100%', height='510px', *args, **kwargs):
    """
    Embeds a folium map in a IPython/Jupyter notebook.
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    fmaps -- a single folium map or an iterable containing folium maps
    """
    from IPython.display import HTML
    template = '<iframe srcdoc="{srcdoc}" style="width: {width}; height: {height}; border: none"></iframe>'
    html = ''
    try:
        for fmap in fmaps:
            fmap._build_map()
            html += template.format(
                srcdoc=fmap.HTML.replace('"', '&quot;'),
                height=str(height),
                width=str(width),
            )
    except TypeError:
        fmap = fmaps
        fmap._build_map()
        html = template.format(
            srcdoc=fmap.HTML.replace('"', '&quot;'),
            height=str(height),
            width=str(width),
        )
        return HTML(html)

In [39]:
map_haer

# Conclusion

think about why and how researchers can actually use data

and why they are confounded (or defeated) at every turn or the amount of time it takes for a staff member to have to explain the shortcomings of the data as it is publicly available (forms, extent, restrictions)